# 13 · GUI Dashboard Demo (SpectraMind V50)

Thin, **CLI‑first** viewer that launches/embeds the generated diagnostics dashboard (HTML) and provides a light local server for browsing artifacts under `outputs/`.

**What this notebook does**
1) Locates a previously generated diagnostics HTML (e.g., `outputs/diagnostics/report.html` or a run‑scoped diagnostics file).
2) (Optional) Calls the CLI to generate the dashboard if missing.
3) Displays the dashboard **in‑notebook** via an iframe.
4) (Optional) Starts a simple local HTTP server to browse `outputs/` (for local dev; disabled on Kaggle).
5) Writes a small viewer **manifest** under `outputs/notebooks/13_gui_dashboard_demo/`.

> Contract: This is a **thin viewer**. No pipeline logic here — we only call the official CLI and render the resulting HTML.


In [ ]:
import os, sys, json, shutil, subprocess, platform, socket, contextlib
from pathlib import Path
from datetime import datetime

ROOT = Path.cwd().resolve()
NB_OUT = ROOT / 'outputs' / 'notebooks' / '13_gui_dashboard_demo'
NB_OUT.mkdir(parents=True, exist_ok=True)

IS_KAGGLE = Path('/kaggle/working').exists()
CLI = shutil.which('spectramind') or (f"{sys.executable} {ROOT/'spectramind.py'}" if (ROOT/'spectramind.py').exists() else f"{sys.executable} -m spectramind")

env = {
    'python': platform.python_version(),
    'platform': platform.platform(),
    'is_kaggle': IS_KAGGLE,
    'cli': CLI
}
(NB_OUT/'env_snapshot.json').write_text(json.dumps(env, indent=2))
print(json.dumps(env, indent=2))

## Parameters
Edit if you need to point at a specific run or force dashboard generation.

In [ ]:
# Where to look for an existing diagnostics HTML
REPORT_HINTS = [
    ROOT/'outputs'/'diagnostics'/'report.html',
    ROOT/'outputs',       # scan recursively for *.html with 'diagnostic' in name
]

# If True, call the CLI to (re)generate the dashboard when not found
AUTO_GENERATE_DASHBOARD = False  # toggle on if you want the notebook to call the CLI

# If generating via CLI, write to this path (adjust per your repo):
CLI_DASHBOARD_OUT = ROOT/'outputs'/'diagnostics'/'report.html'

print('REPORT_HINTS:', [str(p) for p in REPORT_HINTS])
print('AUTO_GENERATE_DASHBOARD:', AUTO_GENERATE_DASHBOARD)
print('CLI_DASHBOARD_OUT:', CLI_DASHBOARD_OUT)

## Locate existing dashboard report
We try direct paths first, then recursively search `outputs/` for files that look like diagnostics dashboards.

In [ ]:
from typing import Optional

def newest_dashboard(hints) -> Optional[Path]:
    cands = []
    for h in hints:
        if not h.exists():
            continue
        if h.is_file() and h.suffix.lower() == '.html':
            cands.append(h)
        elif h.is_dir():
            for p in h.rglob('*.html'):
                name = p.name.lower()
                if any(tok in name for tok in ('diagnostic','dashboard','report')):
                    cands.append(p)
    if not cands:
        return None
    return sorted(cands, key=lambda p: p.stat().st_mtime)[-1]

REPORT = newest_dashboard(REPORT_HINTS)
print('Found dashboard:', REPORT)

## (Optional) Generate the dashboard via CLI
This only runs when `AUTO_GENERATE_DASHBOARD=True`. Adjust the command to match your repository (e.g., `spectramind diagnose dashboard`).

In [ ]:
if AUTO_GENERATE_DASHBOARD:
    try:
        cmd = [
            *CLI.split(), 'diagnose', 'dashboard',
            f'--out', str(CLI_DASHBOARD_OUT)
        ]
        print('Running:', ' '.join(cmd))
        subprocess.run(cmd, check=True)
        REPORT = CLI_DASHBOARD_OUT if CLI_DASHBOARD_OUT.exists() else newest_dashboard([ROOT/'outputs'])
        print('Generated report ->', REPORT)
    except Exception as e:
        print('CLI dashboard generation failed (non‑blocking):', e)
else:
    print('CLI generation disabled; using existing artifacts only.')

## In‑notebook viewer
Renders the dashboard using an iframe. On Kaggle, this is the preferred way (no external ports).

In [ ]:
from IPython.display import IFrame, display, HTML

if REPORT and REPORT.exists():
    # Use relative path when possible so the iframe can resolve asset links
    rel = REPORT.relative_to(ROOT)
    print('Displaying:', rel)
    display(IFrame(src=str(rel), width='100%', height=700))
else:
    print('No diagnostics HTML found. Enable AUTO_GENERATE_DASHBOARD or populate outputs/diagnostics/.')

## (Optional) Lightweight local server (for local dev only)
Starts a simple `http.server` to browse `outputs/`. **Not** recommended on Kaggle (no external ports). Set `START_SERVER=True` only on your workstation.

In [ ]:
START_SERVER = False  # toggle True for local dev only
SERVER_ROOT = ROOT/'outputs'

def free_port(start=8000, end=8999):
    for port in range(start, end+1):
        with contextlib.closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as s:
            if s.connect_ex(('127.0.0.1', port)) != 0:
                return port
    return None

if START_SERVER and not IS_KAGGLE:
    port = free_port()
    if port is None:
        print('No free port found.');
    else:
        print(f'Launching local server at http://127.0.0.1:{port}/ (root={SERVER_ROOT})')
        print('Stop the server by interrupting the cell (Kernel -> Interrupt).')
        os.chdir(SERVER_ROOT)
        try:
            from http.server import ThreadingHTTPServer, SimpleHTTPRequestHandler
        except Exception:
            from http.server import HTTPServer as ThreadingHTTPServer, SimpleHTTPRequestHandler
        handler = SimpleHTTPRequestHandler
        httpd = ThreadingHTTPServer(('127.0.0.1', port), handler)
        try:
            httpd.serve_forever()
        except KeyboardInterrupt:
            print('Shutting down server...')
            httpd.server_close()
else:
    print('Local server disabled (or running on Kaggle).')

## (Optional) Streamlit/Gradio mini‑app
For a richer demo, you can spin up Streamlit/Gradio to serve your plots. This is **off** by default to avoid extra dependencies and network binding issues in shared environments.

Uncomment and adapt the code below if you need it locally.

In [ ]:
USE_STREAMLIT = False
if USE_STREAMLIT and not IS_KAGGLE:
    try:
        import streamlit as st  # ensure installed in your env
    except Exception:
        print('Install streamlit first (pip install streamlit)');
        USE_STREAMLIT = False

if USE_STREAMLIT:
    # Example: write a simple app file and run it
    app_py = NB_OUT/'dashboard_app.py'
    app_py.write_text(
        """
import streamlit as st
from pathlib import Path
st.set_page_config(layout='wide')
st.title('SpectraMind V50 — Diagnostics Dashboard')
report = Path('outputs/diagnostics/report.html')
if report.exists():
    st.components.v1.html(report.read_text(encoding='utf-8'), height=900, scrolling=True)
else:
    st.warning('No diagnostics HTML found under outputs/diagnostics/.')
        """.strip()
    )
    cmd = f"streamlit run {app_py} --server.headless=true"
    print('Launching:', cmd)
    subprocess.run(cmd, shell=True, check=False)
else:
    print('Streamlit mini‑app disabled (or not supported in this environment).')

## Manifest & (optional) DVC add
We save a simple JSON manifest for this viewer session, and optionally `dvc add` the notebook outputs for full reproducibility.

In [ ]:
manifest = {
    'timestamp_utc': datetime.utcnow().isoformat(timespec='seconds')+'Z',
    'report_path': str(REPORT) if REPORT else None,
    'cli': CLI,
    'is_kaggle': IS_KAGGLE
}
(NB_OUT/'viewer_manifest.json').write_text(json.dumps(manifest, indent=2))
print('Wrote:', NB_OUT/'viewer_manifest.json')

if shutil.which('dvc'):
    try:
        subprocess.run(['dvc','add', str(NB_OUT)], check=False)
        subprocess.run(['git','add', f'{NB_OUT}.dvc', '.gitignore'], check=False)
        subprocess.run(['dvc','status'], check=False)
        print('DVC add done (non‑blocking).')
    except Exception as e:
        print('DVC step failed (non‑blocking):', e)
else:
    print('DVC not found; skipping.')